In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image
import pandas as pd
import numpy as np

# --- Custom Dataset ---
class ImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.annotations = csv_file
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(int(self.annotations.iloc[idx, 1]))

        if self.transform:
            image = self.transform(image)

        return image, label

# --- Data Preparation ---
# Load and clean CSV
csv_path = "train.csv"  # Adjust this to the path of your cleaned CSV file
train_df = pd.read_csv(csv_path)
train_df["file_name"] = train_df["file_name"].str.replace("train_data/", "", regex=False)  # Ensure no redundant prefixes
train_df = train_df.drop(columns=["Unnamed: 0"])  # Remove unnecessary column

# Train-validation split
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)

# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets
img_dir = "train_data"  # Directory containing all images
train_dataset = ImageDataset(train_df, img_dir, transform=train_transform)
val_dataset = ImageDataset(val_df, img_dir, transform=val_transform)

# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# --- Define ResNet50 Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)  # Binary classification
model = model.to(device)

# --- Loss and Optimizer ---
criterion = nn.BCEWithLogitsLoss()  # Combines sigmoid and binary cross-entropy
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # Decay learning rate

# --- Training and Validation Loops ---
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation"):
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            all_labels.append(labels.cpu().numpy())
            all_outputs.append(outputs.cpu().numpy())

    epoch_loss = running_loss / len(val_loader.dataset)
    all_labels = np.concatenate(all_labels)
    all_outputs = torch.sigmoid(torch.tensor(np.concatenate(all_outputs))).numpy()  # Convert logits to probabilities

    return epoch_loss, all_labels, all_outputs

# --- Main Training Script ---
num_epochs = 2
best_val_loss = float("inf")

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_labels, val_outputs = validate(model, val_loader, criterion, device)

    scheduler.step()

    # Calculate metrics
    val_preds = (val_outputs > 0.5).astype(int)
    accuracy = (val_preds == val_labels).mean()

    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.4f}")

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_resnet50.pth")
        print("Saved Best Model")

print("Training Complete.")


C:\Users\asens\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\asens\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/2


Validation: 100%|██████████| 250/250 [04:02<00:00,  1.03it/s]


Train Loss: 0.0742, Val Loss: 0.0265, Val Accuracy: 0.9915
Saved Best Model
Epoch 2/2


Validation: 100%|██████████| 250/250 [02:14<00:00,  1.86it/s]

Train Loss: 0.0278, Val Loss: 0.0189, Val Accuracy: 0.9937
Saved Best Model
Training Complete.


In [ ]:
# import os
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torchvision import transforms, models
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# from tqdm import tqdm
# from PIL import Image
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# # --- Custom Dataset ---
# class ImageDataset(Dataset):
#     def __init__(self, csv_file, img_dir, transform=None):
#         self.annotations = csv_file
#         self.img_dir = img_dir
#         self.transform = transform

#     def __len__(self):
#         return len(self.annotations)

#     def __getitem__(self, idx):
#         img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
#         image = Image.open(img_path).convert("RGB")
#         label = torch.tensor(int(self.annotations.iloc[idx, 1]))

#         if self.transform:
#             image = self.transform(image)

#         return image, label

# # --- Early Stopping ---
# class EarlyStopping:
#     def __init__(self, patience=5, verbose=True):
#         """
#         Early stopping to stop training when validation loss doesn't improve.
#         Args:
#         - patience (int): Number of epochs to wait for improvement.
#         - verbose (bool): Whether to print messages.
#         """
#         self.patience = patience
#         self.verbose = verbose
#         self.counter = 0
#         self.best_loss = None
#         self.early_stop = False

#     def __call__(self, val_loss, model, path="best_model.pth"):
#         if self.best_loss is None or val_loss < self.best_loss:
#             self.best_loss = val_loss
#             self.counter = 0
#             torch.save(model.state_dict(), path)
#             if self.verbose:
#                 print(f"Validation loss improved to {val_loss:.4f}. Model saved!")
#         else:
#             self.counter += 1
#             if self.verbose:
#                 print(f"No improvement. EarlyStopping counter: {self.counter}/{self.patience}")
#             if self.counter >= self.patience:
#                 self.early_stop = True

# # --- Data Preparation ---
# # Load and clean CSV
# csv_path = "train.csv"  # Adjust this to the path of your cleaned CSV file
# train_df = pd.read_csv(csv_path)
# train_df["file_name"] = train_df["file_name"].str.replace("train_data/", "", regex=False)  # Ensure no redundant prefixes
# train_df = train_df.drop(columns=["Unnamed: 0"])  # Remove unnecessary column

# # Train-validation split
# train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)

# # Define transformations
# train_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(10),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# val_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# # Create datasets
# img_dir = "train_data"  # Directory containing all images
# train_dataset = ImageDataset(train_df, img_dir, transform=train_transform)
# val_dataset = ImageDataset(val_df, img_dir, transform=val_transform)

# # Create dataloaders
# batch_size = 64
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# # --- Define ResNet50 Model ---
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = models.resnet50(pretrained=True)
# num_features = model.fc.in_features
# model.fc = nn.Linear(num_features, 1)  # Binary classification
# model = model.to(device)

# # --- Loss and Optimizer ---
# criterion = nn.BCEWithLogitsLoss()  # Combines sigmoid and binary cross-entropy
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # Decay learning rate

# # --- Training and Validation Functions ---
# def train_one_epoch(model, train_loader, criterion, optimizer, device):
#     model.train()
#     running_loss = 0.0

#     for images, labels in tqdm(train_loader, desc="Training"):
#         images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item() * images.size(0)

#     epoch_loss = running_loss / len(train_loader.dataset)
#     return epoch_loss

# def validate(model, val_loader, criterion, device):
#     model.eval()
#     running_loss = 0.0
#     all_labels = []
#     all_outputs = []

#     with torch.no_grad():
#         for images, labels in tqdm(val_loader, desc="Validation"):
#             images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

#             outputs = model(images)
#             loss = criterion(outputs, labels)

#             running_loss += loss.item() * images.size(0)
#             all_labels.append(labels.cpu().numpy())
#             all_outputs.append(outputs.cpu().numpy())

#     epoch_loss = running_loss / len(val_loader.dataset)
#     all_labels = np.concatenate(all_labels)
#     all_outputs = torch.sigmoid(torch.tensor(np.concatenate(all_outputs))).numpy()  # Convert logits to probabilities

#     return epoch_loss, all_labels, all_outputs

# # --- Main Training Script ---
# num_epochs = 20
# best_val_loss = float("inf")
# early_stopping = EarlyStopping(patience=5, verbose=True)

# # Initialize lists to store metrics
# train_losses = []
# val_losses = []
# val_accuracies = []

# # Plotting setup
# plt.ion()
# fig, ax = plt.subplots()

# for epoch in range(num_epochs):
#     print(f"Epoch {epoch+1}/{num_epochs}")

#     # Train for one epoch
#     train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
#     train_losses.append(train_loss)

#     # Validate after each epoch
#     val_loss, val_labels, val_outputs = validate(model, val_loader, criterion, device)
#     val_losses.append(val_loss)

#     # Calculate Validation Accuracy
#     val_preds = (val_outputs > 0.5).astype(int)
#     accuracy = (val_preds == val_labels).mean()
#     val_accuracies.append(accuracy)

#     # Print metrics for this epoch
#     print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.4f}")

#     # Update learning rate
#     scheduler.step()

#     # Early stopping
#     early_stopping(val_loss, model, path="best_resnet501.pth")
#     if early_stopping.early_stop:
#         print("Early stopping triggered!")
#         break

#     # Update live plot
#     ax.clear()
#     ax.plot(range(1, len(train_losses) + 1), train_losses, label="Train Loss", marker="o")
#     ax.plot(range(1, len(val_losses) + 1), val_losses, label="Validation Loss", marker="o")
#     ax.set_xlabel("Epochs")
#     ax.set_ylabel("Loss")
#     ax.set_title("Training and Validation Loss")
#     ax.legend()
#     plt.draw()
#     plt.pause(0.01)

# # Save metrics for analysis
# metrics = {
#     "train_loss": train_losses,
#     "val_loss": val_losses,
#     "val_accuracy": val_accuracies
# }

# with open("training_metrics.json", "w") as f:
#     json.dump(metrics, f)
# print("Saved training metrics to 'training_metrics.json'")

# plt.ioff()
# plt.show()

# print("Training Complete.")


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

# --- Custom Dataset ---
class ImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.annotations = csv_file
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(int(self.annotations.iloc[idx, 1]))

        if self.transform:
            image = self.transform(image)

        return image, label

# --- Early Stopping ---
class EarlyStopping:
    def __init__(self, patience=5, verbose=True):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss, model, optimizer, scheduler, epoch, path="checkpoint.pth"):
        if self.best_loss is None or val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
            torch.save({
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "scheduler_state_dict": scheduler.state_dict(),
                "best_loss": self.best_loss
            }, path)
            if self.verbose:
                print(f"Validation loss improved to {val_loss:.4f}. Checkpoint saved!")
        else:
            self.counter += 1
            if self.verbose:
                print(f"No improvement. EarlyStopping counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True

# --- Data Preparation ---
csv_path = "train.csv"
train_df = pd.read_csv(csv_path)
train_df["file_name"] = train_df["file_name"].str.replace("train_data/", "", regex=False)
train_df = train_df.drop(columns=["Unnamed: 0"])

train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img_dir = "train_data"
train_dataset = ImageDataset(train_df, img_dir, transform=train_transform)
val_dataset = ImageDataset(val_df, img_dir, transform=val_transform)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# --- Model Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# --- Resume Training ---
def load_checkpoint(path, model, optimizer, scheduler):
    if os.path.exists(path):
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
        start_epoch = checkpoint["epoch"] + 1
        best_val_loss = checkpoint["best_loss"]
        print(f"Checkpoint loaded: Resuming from epoch {start_epoch} with best_val_loss {best_val_loss:.4f}")
        return start_epoch, best_val_loss
    else:
        print("No checkpoint found. Starting from scratch.")
        return 0, float("inf")

# --- Training and Validation Functions ---
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation"):
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            all_labels.append(labels.cpu().numpy())
            all_outputs.append(outputs.cpu().numpy())

    epoch_loss = running_loss / len(val_loader.dataset)
    all_labels = np.concatenate(all_labels)
    all_outputs = torch.sigmoid(torch.tensor(np.concatenate(all_outputs))).numpy()

    return epoch_loss, all_labels, all_outputs

# --- Main Training Script ---
num_epochs = 20
checkpoint_path = "checkpoint.pth"
start_epoch, best_val_loss = load_checkpoint(checkpoint_path, model, optimizer, scheduler)
early_stopping = EarlyStopping(patience=5, verbose=True)

train_losses = []
val_losses = []
val_accuracies = []

plt.ion()
fig, ax = plt.subplots()

for epoch in range(start_epoch, num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)

    val_loss, val_labels, val_outputs = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)

    val_preds = (val_outputs > 0.5).astype(int)
    accuracy = (val_preds == val_labels).mean()
    val_accuracies.append(accuracy)

    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.4f}")

    scheduler.step()

    early_stopping(val_loss, model, optimizer, scheduler, epoch, path=checkpoint_path)
    if early_stopping.early_stop:
        print("Early stopping triggered!")
        break

    ax.clear()
    ax.plot(range(1, len(train_losses) + 1), train_losses, label="Train Loss", marker="o")
    ax.plot(range(1, len(val_losses) + 1), val_losses, label="Validation Loss", marker="o")
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Loss")
    ax.set_title("Training and Validation Loss")
    ax.legend()
    plt.draw()
    plt.pause(0.01)

metrics = {
    "train_loss": train_losses,
    "val_loss": val_losses,
    "val_accuracy": val_accuracies
}

with open("training_metrics.json", "w") as f:
    json.dump(metrics, f)
print("Saved training metrics to 'training_metrics.json'")

plt.ioff()
plt.show()

print("Training Complete.")


In [ ]:
# class GradCAM:
#     def __init__(self, model, target_layer):
#         """
#         Initialize Grad-CAM with the model and the target layer.
#         Args:
#         - model: Trained PyTorch model.
#         - target_layer: Layer to compute Grad-CAM for.
#         """
#         self.model = model
#         self.target_layer = target_layer
#         self.gradients = None

#         # Hook to get gradients
#         target_layer.register_backward_hook(self.save_gradients)

#     def save_gradients(self, module, grad_input, grad_output):
#         self.gradients = grad_output[0]

#     def generate_heatmap(self, feature_maps, gradients):
#         """
#         Generate a heatmap using feature maps and gradients.
#         """
#         weights = gradients.mean(dim=(2, 3), keepdim=True)  # Global average pooling
#         cam = (weights * feature_maps).sum(dim=1, keepdim=True)  # Weighted sum
#         cam = torch.relu(cam)  # ReLU to keep only positive values
#         cam = cam.squeeze().cpu().detach().numpy()
#         cam = (cam - cam.min()) / (cam.max() - cam.min())  # Normalize to [0, 1]
#         return cam

#     def __call__(self, input_image, target_class):
#         """
#         Compute Grad-CAM for a given input and target class.
#         Args:
#         - input_image: Preprocessed input tensor of shape [1, C, H, W].
#         - target_class: Class index for which Grad-CAM is computed.
#         """
#         self.model.eval()

#         # Forward pass
#         feature_maps = None
#         for name, module in self.model.named_modules():
#             input_image = module(input_image)
#             if name == self.target_layer:
#                 feature_maps = input_image

#         # Backward pass
#         self.model.zero_grad()
#         output = input_image[:, target_class]
#         output.backward()

#         # Generate heatmap
#         gradients = self.gradients
#         heatmap = self.generate_heatmap(feature_maps, gradients)
#         return heatmap


In [ ]:
# from torchvision.transforms.functional import to_pil_image
# import cv2

# def apply_gradcam(model, image_path, target_layer, label_map):
#     """
#     Apply Grad-CAM to a single image.
#     Args:
#     - model: Trained PyTorch model.
#     - image_path: Path to the input image.
#     - target_layer: Target layer for Grad-CAM.
#     - label_map: Dictionary mapping labels to class names.
#     """
#     transform = transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ])

#     # Load and preprocess image
#     image = Image.open(image_path).convert("RGB")
#     input_tensor = transform(image).unsqueeze(0).to(device)

#     # Get prediction
#     with torch.no_grad():
#         output = model(input_tensor)
#         pred_class = torch.sigmoid(output).item() > 0.5  # Binary prediction
#         pred_label = label_map[int(pred_class)]

#     # Apply Grad-CAM
#     gradcam = GradCAM(model, target_layer)
#     heatmap = gradcam(input_tensor, int(pred_class))

#     # Visualize heatmap
#     heatmap_resized = cv2.resize(heatmap, (224, 224))
#     heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET)
#     overlay = cv2.addWeighted(np.array(image), 0.5, heatmap_colored, 0.5, 0)

#     plt.figure(figsize=(10, 5))
#     plt.subplot(1, 2, 1)
#     plt.imshow(image)
#     plt.title(f"Original Image ({pred_label})")
#     plt.axis("off")

#     plt.subplot(1, 2, 2)
#     plt.imshow(overlay)
#     plt.title("Grad-CAM Heatmap")
#     plt.axis("off")

#     plt.tight_layout()
#     plt.show()

# # Example Usage
# label_map = {0: "Real", 1: "Fake"}
# image_path = "path/to/test/image.jpg"
# apply_gradcam(model, image_path, "layer4", label_map)
